In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Kernel Methods/')

Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd
from time import time
import cvxopt
import sys

In [3]:
class Helper:
    
    @staticmethod
    def log_process(title, cursor, finish_cursor, start_time = None):
        percentage = float(cursor + 1)/finish_cursor
        now_time = time()
        time_to_finish = ((now_time - start_time)/percentage) - (now_time - start_time)
        mn, sc = int(time_to_finish//60), int((time_to_finish/60 - time_to_finish//60)*60)
        if start_time:
            sys.stdout.write("\r%s - %.2f%% ----- Temps restant estimé: %d min %d sec -----" %(title, 100*percentage, mn, sc))
            sys.stdout.flush()
        else:
            sys.stdout.write("\r%s - \r%.2f%%" %(title, 100*percentage))
            sys.stdout.flush()

In [4]:
class SIFT:

    ###################
    #    CONSTUCTOR   #
    ###################

    def __init__(self, gaussian_thres = 1.0, gaussian_sigma = 0.8, sift_thres = 0.2, \
                 num_angles = 12, num_bins = 5, alpha = 9.0):
        self.num_angles = num_angles
        self.num_bins = num_bins
        self.alpha = alpha
        self.angle_list = np.array(range(num_angles))*2.0*np.pi/num_angles
        self.gaussian_thres = gaussian_thres
        self.gaussian_sigma = gaussian_sigma
        self.sift_thres = sift_thres
        self.weights = self._get_weights(num_bins)

    ###################
    # PUBLIC  METHODS #
    ###################
    
    def get_features(self, data):
        out = []
        start = time()
        finish = len(data)
        for idx, dt in enumerate(data):
            Helper.log_process('SIFT', idx, finish_cursor=finish, start_time = start)
            feature = self._calculate_sift_feature(np.mean(np.double(dt), axis=2))
            feature = self._normalize_sift(feature)
            out.append(feature)
        return np.array(out)

    
    ###################
    # PRIVATE METHODS #
    ###################

    def _get_weights(self, num_bins):
        size_unit = np.array(range(32))
        sph, spw = np.meshgrid(size_unit, size_unit)
        sph.resize(sph.size)
        spw.resize(spw.size)
        bincenter = np.array(range(1, num_bins*2, 2)) / 2.0 / num_bins * 32 - 0.5
        bincenter_h, bincenter_w = np.meshgrid(bincenter, bincenter)
        bincenter_h.resize((bincenter_h.size, 1))
        bincenter_w.resize((bincenter_w.size, 1))
        dist_ph = abs(sph - bincenter_h)
        dist_pw = abs(spw - bincenter_w)
        weights_h = dist_ph / (32 / np.double(num_bins))
        weights_w = dist_pw / (32 / np.double(num_bins))
        weights_h = (1-weights_h) * (weights_h <= 1)
        weights_w = (1-weights_w) * (weights_w <= 1)
        return weights_h * weights_w

    def _calculate_sift_feature(self, image):
        if image.ndim == 3:
            image = np.mean(image, axis=2)
        H, W = image.shape
        features = np.zeros((1, self.num_bins * self.num_bins * self.num_angles))
        gaussian_height, gaussian_width = self._get_gauss_filter(self.gaussian_sigma)
        IH = self._convolution2D(image, gaussian_height)
        IW = self._convolution2D(image, gaussian_width)
        Imag = np.sqrt(IH**2 + IW**2)
        Itheta = np.arctan2(IH,IW)
        Iorient = np.zeros((self.num_angles, H, W))
        for i in range(self.num_angles):
            Iorient[i] = Imag * np.maximum(np.cos(Itheta - self.angle_list[i])**self.alpha, 0)
        feature = np.zeros((self.num_angles, self.num_bins**2))
        for j in range(self.num_angles):
             feature[j] = np.dot(self.weights,\
                     Iorient[j,:,:].flatten())
        feature = feature.flatten()
        return feature

    def _normalize_sift(self, feature):
        siftlen = np.sqrt(np.sum(feature**2))
        test = (siftlen >= self.gaussian_thres)
        if siftlen < self.gaussian_thres:
          siftlen = self.gaussian_thres
        feature /= siftlen
        feature[feature>self.sift_thres] = self.sift_thres
        if test:
            feature /= np.sqrt(np.sum(feature**2))
        return feature


    def _get_gauss_filter(self, sigma):
        gaussian_filter_amp = np.int(2*np.ceil(sigma))
        gaussian_filter = np.array(range(-gaussian_filter_amp, gaussian_filter_amp+1))**2
        gaussian_filter = gaussian_filter[:, np.newaxis] + gaussian_filter
        gaussian_filter = np.exp(- gaussian_filter / (2.0 * sigma**2))
        gaussian_filter /= np.sum(gaussian_filter)
        gaussian_height, gaussian_width = np.gradient(gaussian_filter)
        gaussian_height *= 2.0/np.sum(np.abs(gaussian_height))
        gaussian_width  *= 2.0/np.sum(np.abs(gaussian_width))
        return gaussian_height, gaussian_width
    
    def _convolution2D(self, image, kernel):
        imRows, imCols = image.shape
        kRows, kCols = kernel.shape

        y = np.zeros((imRows,imCols))

        kcenterX = kCols//2
        kcenterY = kRows//2

        for i in range(imRows):
            for j in range(imCols):
                for m in range(kRows):
                    mm = kRows - 1 - m
                    for n in range(kCols):
                        nn = kCols - 1 - n

                        ii = i + (m - kcenterY)
                        jj = j + (n - kcenterX)

                        if ii >= 0 and ii < imRows and jj >= 0 and jj < imCols :
                            y[i][j] += image[ii][jj] * kernel[mm][nn]

        return y

In [5]:
class Kernel:
    
    ########################
    # Only static  methods #
    ########################
    
    @staticmethod
    def RBF(sigma):
        def f(X1, X2):
            squared_norm = np.expand_dims(np.sum(X1**2,axis=1),axis=1) + np.expand_dims(np.sum(X2**2,axis=1),axis=0)-2*np.einsum('ni,mi->nm',X1,X2)
            return np.exp(-0.5*squared_norm/sigma**2)
        return f

    @staticmethod
    def linear():
        def f(X1, X2):
            return X1.dot(X2.T)
        return f

    @staticmethod
    def polynomial(degree):
        def f(X1, X2):
            return X1.dot(X2.T) ** degree
        return f
    
    @staticmethod
    def chi2(gamma):
        def f(X1, X2):
            out = np.zeros((X1.shape[0], X2.shape[0]))
            n_X1 = X1.shape[0]
            n_X2 = X2.shape[0]
            n_features = X1.shape[1]

            for i in range(n_X1):
                for j in range(n_X2):
                    p = 0
                    for k in range(n_features):
                        denominateur = (X1[i, k] - X2[j, k])
                        nominateur = (X1[i, k] + X2[j, k])
                        if nominateur != 0:
                            p += denominateur * denominateur / nominateur
                    out[i, j] = -p
            tmp = gamma * out
            return  np.exp(tmp, tmp)        
        return f

In [6]:
class SVC:
    
    ###############
    # Constructor #
    ###############
    
    def __init__(self, C=1.0, kernel='linear', gamma=.6, sigma=1., degree=2):
        self.C = C
        self.degree = degree
        self.gamma = gamma
        self.sigma = sigma
        self.kernel = self._get_kernel(kernel, gamma=gamma, sigma=sigma, degree=degree)
    
    ##################
    # Public methods #
    ##################
    
    def fit(self, X, y):
        self._X, self._y = X, y
        
        self.labels = np.unique(y)
        self.n_labels = len(self.labels)
        self._K = self.kernel(X, X)
        
        # OneVsAll
        models = {}
        start = time()
        finish = len(self.labels)
        for idx, label in enumerate(self.labels):

            models[label] = {}
            y_label = np.array([1. if e == label else -1. for e in y])
            w, b, mu_support, idx_support = self._fit_binary(X, y_label)
            
            models[label]['y'] = y_label
            models[label]['w'] = w
            models[label]['b'] = b  
            models[label]['mu_support'] = mu_support
            models[label]['idx_support'] = idx_support
            
            Helper.log_process('SVC Fitting', idx, finish_cursor=finish, start_time = start)
            
        self.models = models
    
    def predict(self, X):
        scores = [self._predict(X, self.models[label]['y'], self.models[label]['idx_support'], self.models[label]['mu_support'], self.models[label]['b']) for label in self.labels]
        return self.labels[np.argmax(np.array(scores), axis=0)]

    def next_layer(self, X):
        scores = [self._predict(X, self.models[label]['y'], self.models[label]['idx_support'], self.models[label]['mu_support'], self.models[label]['b']) for label in self.labels]
        return np.array(scores).T
    
    ###################
    # Private methods #
    ###################
        
    def _qp(self, H, e, A, b, C=np.inf, l=1e-8, verbose=True):
        # Gram matrix
        n = H.shape[0]
        H = cvxopt.matrix(H)
        A = cvxopt.matrix(A, (1, n))
        e = cvxopt.matrix(-e)
        b = cvxopt.matrix(0.0)
        if C == np.inf:
            G = cvxopt.matrix(np.diag(np.ones(n) * -1))
            h = cvxopt.matrix(np.zeros(n))
        else:
            G = cvxopt.matrix(np.concatenate([np.diag(np.ones(n) * -1),
                                             np.diag(np.ones(n))], axis=0))
            h = cvxopt.matrix(np.concatenate([np.zeros(n), C * np.ones(n)]))

        # Solve QP problem
        cvxopt.solvers.options['show_progress'] = verbose
        solution = cvxopt.solvers.qp(H, e, G, h, A, b)

        # Lagrange multipliers
        mu = np.ravel(solution['x'])
        return mu
    
    def _predict(self, X, y_model, idx_support, mu_support, b):
        X_support = self._X[idx_support]
        G = self.kernel(X, X_support)
        return G.dot(mu_support * y_model[idx_support]) + b
    
    def _fit_binary(self, X, y):
        mu_support, idx_support = self._svm_solver_non_sep(self._K, y, self.C)
        w = self._get_w(mu_support, idx_support, X, y)
        b = self._compute_b(self._K, y, mu_support, idx_support)
        return w, b, mu_support, idx_support
    
    def _svm_solver_non_sep(self, K, y, C):
        n = y.shape[0]
        y = y.reshape((n, 1))
        H = np.dot(y, y.T)*K
        e = np.ones(n)
        A = y
        b = np.zeros(n)
        mu = self._qp(H, e, A, b, C, l=1e-8, verbose=False)
        idx_support = np.where(np.abs(mu) > 1e-5)[0]
        mu_support = mu[idx_support]
        return mu_support, idx_support
    
    def _get_w(self, mu_support, idx_support, X, y):
        return np.sum((mu_support * y[idx_support])[: , None] * X[idx_support], axis=0)
    
    def _get_kernel(self, kernel, gamma, sigma, degree):
        return {
            'linear': Kernel.linear(),
            'polynomial': Kernel.polynomial(degree),
            'chi2': Kernel.chi2(gamma),
            'RBF': Kernel.RBF(sigma)
        }[kernel]

    def _compute_b(self, K, y, mu_support, idx_support):
        num_support_vector = idx_support.size
        y_support = y[idx_support]
        K_support = K[idx_support][:, idx_support]
        b = [y_support[j] - sum([mu_support[i]*y_support[i]*K_support[i][j] for i in range(num_support_vector)]) for j in range(num_support_vector)]
        return np.mean(b)

In [8]:
X_df = pd.read_csv('./Xtr.csv', header=None)
y_df = pd.read_csv('./Ytr.csv')
X_df = X_df.loc[:5000,:3071]
y_df=y_df.loc[:5000]

X_test = pd.read_csv('./Xte.csv', header=None)
X_test = X_test.loc[:,:3071]

X = X_df.values
y = y_df.Prediction

X_test = X_test.values

In [9]:
red, green, blue = np.hsplit(X, 3)
data = np.array([np.dstack((red[i], blue[i], green[i])).reshape(32, 32, 3) for i in range(len(X))])

red, green, blue = np.hsplit(X_test, 3)
data_test = np.array([np.dstack((red[i], blue[i], green[i])).reshape(32, 32, 3) for i in range(len(X_test))])

In [10]:
params = { 'chi2_gamma': .6,
           'sigma': 1.,
           'degree': 2.,
           'C': 10.,
           'sift_thres': .3,
           'gaussian_thres': .7,
           'gaussian_sigma': .4,
           'num_angles': 12,
           'num_bins': 5,
           'alpha': 9.0 }

In [11]:
extractor = SIFT(sift_thres=params['sift_thres'], 
                 gaussian_sigma=params['gaussian_sigma'], 
                 gaussian_thres=params['gaussian_thres'],
                 num_angles=params['num_angles'],
                 num_bins=params['num_bins'],
                 alpha=params['alpha'])

In [12]:
X_train = extractor.get_features(data)

SIFT - 1.98% ----- Temps restant estimé: 0 min 5 sec -----

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


SIFT - 100.00% ----- Temps restant estimé: 0 min 0 sec -----

In [13]:
clf1 = SVC(kernel='RBF', C=params['C'], sigma=params['sigma'])
clf2 = SVC(kernel='RBF', C=params['C'], sigma=params['sigma'])

In [14]:
clf1.fit(X_train[:,:], y[:])

SVC Fitting - 100.00% ----- Temps restant estimé: 0 min 0 sec -----

In [15]:
layer2 = clf1.next_layer(X_train[:5000,:])
clf2.fit(layer2, y[:5000])

SVC Fitting - 100.00% ----- Temps restant estimé: 0 min 0 sec -----

In [17]:
X_test = extractor.get_features(data_test)

SIFT - 3.00% ----- Temps restant estimé: 0 min 6 sec -----

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


SIFT - 100.00% ----- Temps restant estimé: 0 min 0 sec -----

In [ ]:
X_test_layer2 = clf1.next_layer(X_test)
pred = clf2.predict(X_test_layer2)
predictions = pd.DataFrame(pred, columns=['Prediction'])
predictions['id'] = np.arange(1, len(predictions)+1)
predictions.to_csv('submission_RBF_2layers.csv', sep=',', index=False, encoding='utf-8', columns=['id', 'Prediction'])

In [ ]:
clf3 = SVC(kernel='polynomial', C=params['C'], gamma=params['chi2_gamma'], sigma=params['sigma'])

In [ ]:
clf3.fit(layer2, y[:5000])

SVC Fitting - 100.00% ----- Temps restant estimé: 0 min 0 sec -----

In [ ]:
pred = clf3.predict(X_test_layer2)
predictions = pd.DataFrame(pred, columns=['Prediction'])
predictions['id'] = np.arange(1, len(predictions)+1)
predictions.to_csv('submission_RBF_polynomial_2layers.csv', sep=',', index=False, encoding='utf-8', columns=['id', 'Prediction'])